## Getting the data


### Getting data

## Building a tokenizer


In [ ]:
!pip install pathlib

In [ ]:
from pathlib import Path #it will be used to get all the file names

pths = [str(i) for i in Path('../input/sanskrit-text-corpus/Sankrit_Corpus/').glob('*.txt')]
#print(pths)

In [ ]:
!pip install tokenizers

In [ ]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

In [ ]:
tokenizer.train(files=pths, vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [ ]:
#!mkdir 'content/tok''<s>', '<pad>', '</s>', '<unk>', '<mask>'

In [ ]:
import os
os.mkdir('./sanskrit_bert')
tokenizer.save_model('sanskrit_bert')

In [ ]:
print(tokenizer)

In [ ]:
!pip install transformers

In [ ]:
from transformers import RobertaTokenizer

# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('./sanskrit_bert', max_len=512)

In [ ]:
tokens = tokenizer('अस्याः सञ्चिकायाः')
print(tokens)
tokens = tokenizer('इहापि अनन्तरस्य पितृष्वसुर्विधिर्वा भवतिप्रतिषेधो वा इति लोप एवानन्तरः प्राप्नोति अस्याः')
print(tokens)
t = 'ॐ नमो भगवते॑ रुद्राय ॥ नम॑स्ते रुद्र म॒न्यव॑ उ॒तोत॒ इष॑वे॒ नमः॑ । नम॑स्ते अस्तु॒ धन्व॑ने बा॒हुभ्या॑मु॒त ते॒ नमः॑ '
tokens = tokenizer(t)
print(tokens)
len(tokens)

In [ ]:
import glob
lines=[]

In [ ]:
for file1 in glob.glob("../input/sanskrit-text-corpus/Sankrit_Corpus/*.txt"):
  with open(file1, 'r', encoding='utf-8') as fp:
      lines+=(fp.read().split('\n'))

In [ ]:
len(lines)
print(lines[230451])
lines = lines[int(len(lines)*0.55):int(len(lines)*0.85)]

In [ ]:
int(len(lines)*0.2//1)

In [ ]:
batch = tokenizer(lines[:], max_length=300, padding='max_length', truncation=True)


In [ ]:
len(batch["input_ids"])

In [ ]:

for x in batch:
  print(x)

In [ ]:
import torch

labels = torch.tensor([x for x in batch["input_ids"]])
mask = torch.tensor([x for x in batch["attention_mask"]])

In [ ]:
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3

In [ ]:
input_ids.shape

In [ ]:
input_ids[100][:]

In [ ]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels}


In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [ ]:
dataset = Dataset(encodings)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

In [ ]:
model


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
epochs = 10

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    model.save_pretrained('./sanskrit_bert')     

In [ ]:
model.save_pretrained('./sanskrit_bert')  

In [ ]:
model

In [ ]:
from transformers import pipeline

## Creating an input pipeline


In [ ]:
fill = pipeline('fill-mask', model='sanskrit_bert', tokenizer='sanskrit_bert')

In [ ]:
fill(f'पतत्र श्रीरामं {fill.tokenizer.mask_token} पहिता')


###loading model

In [ ]:
from transformers import RobertaTokenizer, RobertaModel

In [ ]:
model = RobertaModel.from_pretrained("sanskrit_bert")

model.eval()

In [ ]:
atokenizer = RobertaTokenizer.from_pretrained("sanskrit_bert")

In [ ]:
def get_sen_emb(sen):
    try:
        enc = atokenizer.encode(sen)
        ps = model(torch.tensor(enc).unsqueeze(1))
        sentence_embedding = torch.mean(ps[1], dim=0) #mean of all tokens embeddings will be consideres as sentence embedding
        s=sentence_embedding.cpu().detach().numpy()
        s=s.reshape(s.shape[0])
        return(s)

    except:
        print(1)
        if(atokenizer.encode(sen)>500):
               print(sen)
        
        

def plot_embd(em_np):
    plt.hist(em_np,edgecolor="yellow",bins=20)
    plt.show()

In [ ]:
s="""'ॐ नमो भगवते॑ रुद्राय ॥ नम॑स्ते रुद्र म॒न्यव॑ उ॒तोत॒ इष॑वे॒ नमः॑ । नम॑स्ते अस्तु॒ धन्व॑ने बा॒हुभ्या॑मु॒त ते॒ नमः॑ । या त॒ इषुः॑ शि॒वत॑मा शि॒वं ब॒भूव॑ ते॒ धनुः॑ । शि॒वा श॑र॒व्या॑ या तव॒ तया॑ नो रुद्र मृडय । या ते॑ रुद्र शि॒वा त॒नूरघो॒रा$पा॑पकाशिनी । तया॑ नस्त॒नुवा॒ शन्त॑मया॒ गिरि॑शन्ता॒भिचा॑कशीहि । यामिषुं॑ गिरिशन्त॒ हस्ते॒ बिभ॒र्ष्यस्त॑वे । शि॒वां गि॑रित्र॒ तां कु॑रु॒ मा हिग्ं॑सीः॒ पुरु॑षं॒ जग॑त्। शि॒वेन॒ वच॑सा त्वा॒ गिरि॒शाच्छा॑वदामसि । यथा॑ नः॒ सर्व॒मिज्जग॑दय॒क्ष्मग्ं सु॒मना॒ अस॑त् । अध्य॑वोचदधिव॒क्ता प्र॑थ॒मो दैव्यो॑ भि॒षक् । अहीश्च॒ सर्वां॓ज॒म्भय॒न्त्सर्वा॓श्च यातुधा॒न्यः॑ ।अ॒सौ यस्ता॒म्रो अ॑रु॒ण उ॒त ब॒भ्रुः सु॑म॒ङ्गलः॑ । ये चे॒माग्ं रु॒द्रा अ॒भितो॑ दि॒क्षु श्रि॒ताः स॑हस्र॒शोवैषा॒ग्॒ं॒ हेड॑ ईमहे । अ॒सौ यो॑$वसर्प॑ति॒ नील॑ग्रीवो॒ विलो॑हितः । उ॒तैनं॑ गो॒पा अ॑दृश॒न्-नदृ॑शन्-नुदहा॒र्यः॑ । उ॒तैनं॒ विश्वा॑ भू॒तानि॒ स दृ॒ष्टो मृ॑डयाति नः । नमो॑ अस्तु नील॑ग्रीवाय सहस्रा॒क्षाय॒ मी॒ढुषे॓ । अथो॒ ये अ॑स्य॒ सत्वा॑नो॒$हं तेभ्यो॑$कर॒न्नमः॑ । प्रमुं॑च॒ धन्व॑न॒स्-त्व॒मु॒भयो॒रार्त्नि॑ यो॒र्ज्याम् । याश्च ते॒ हस्त॒ इष॑वः॒ परा॒ ता भ॑गवो वप । अ॒व॒तत्य॒ धनु॒स्त्वग्ं सह॑स्राक्ष॒ शते॑षुधे । नि॒शीर्य॑ श॒ल्यानां॒ मुखा॑ शि॒वो नः॑ सु॒मना॑ भव । विज्यं॒ धनुः॑ कप॒र्दिनो॒ विश॑ल्यो॒ बाण॑वाग्म् उ॒त । अने॑श॒न्-नस्येष॑व आ॒भुर॑स्य निष॒ङ्गथिः॑ ।या ते॑ हे॒तिर्-मी॑डुष्टम॒ हस्ते॑ ब॒भूव॑ ते॒ धनुः॑ । तया॒$स्मान्, वि॒श्वत॒स्-त्वम॑य॒क्ष्मया॒ परि॑ब्भुज । नम॑स्ते अ॒स्त्वायुधा॒याना॑तताय धृ॒ष्णवे॓ । उ॒भाभ्या॑मु॒त ते॒ नमो॑ बा॒हुभ्यां॒ तव॒ धन्व॑ने । परि॑ ते॒ धन्व॑नो हे॒तिर॒स्मान्-वृ॑णक्तु वि॒श्वतः॑ । अथो॒ य इ॑षु॒धिस्तवा॒रे अ॒स्मन्निधे॑हि॒ तम् ॥ 1 ॥नम॑स्ते अस्तु भगवन्-विश्वेश्व॒राय॑ महादे॒वाय॑ त्र्यम्ब॒काय॑ त्रिपुरान्त॒काय॑ त्रिकाग्निका॒लाय॑ कालाग्निरु॒द्राय॑ नील॒कण्ठाय॑ मृत्युञ्ज॒याय॑ सर्वेश्व॑राय॑ सदाशि॒वाय॑ श्रीमन्-महादे॒वाय॒ नमः॑ ॥नमो॒ हिर॑ण्य बाहवे सेना॒न्ये॑ दि॒शां च॒ पत॑ये॒ नमो॒ नमो॑ वृ॒क्षेभ्यो॒ हरि॑केशेभ्यः पशू॒नां पत॑ये॒ नमो॒ नमः॑ स॒स्पिञ्ज॑राय॒ त्विषी॑मते पथी॒नां पत॑ये॒ नमो॒ नमो॑ बभ्लु॒शाय॑ विव्या॒धिने$न्ना॑नां॒ पत॑ये॒ नमो॒ नमो॒ हरि॑केशायोपवी॒तिने॑ पु॒ष्टानां॒ पत॑ये॒ नमो॒ नमो॑ भ॒वस्य॑ हे॒त्यै जग॑तां॒ पत॑ये॒ नमो॒ नमो॑ रु॒द्राया॑तता॒विने॒ क्षेत्रा॑णां॒ पत॑ये॒ नमो॒ नमः॑ सू॒तायाहं॑त्याय॒ वना॑नां॒ पत॑ये॒ नमो॒ नमो॒ रोहि॑ताय स्थ॒पत॑ये वृ॒क्षाणां॒ पत॑ये॒ नमो॒ नमो॑ म॒न्त्रिणे॑ वाणि॒जाय॒ कक्षा॑णां॒ पत॑ये॒ नमो॒ नमो॑ भुव॒न्तये॑ वारिवस्कृ॒ता-यौष॑धीनां॒ पत॑ये॒ नमो॒ नम॑ उ॒च्चैर्-घो॑षायाक्र॒न्दय॑ते पत्ती॒नां पत॑ये॒ नमो॒ नमः॑ कृत्स्नवी॒ताय॒ धाव॑ते॒ सत्त्व॑नां॒ पत॑ये॒ नमः॑ नमः॒ सह॑मानाय निव्या॒धिन॑ आव्या॒धिनी॑नां॒ पत॑ये नमो॒ नमः॑ ककु॒भाय॑ निष॒ङ्गिणे॓ स्ते॒नानां॒ पत॑ये॒ नमो॒ नमो॑ निष॒ङ्गिण॑ इषुधि॒मते॑ तस्क॑राणां॒ पत॑ये॒ नमो॒ नमो॒ वञ्च॑ते परि॒वञ्च॑ते स्तायू॒नां पत॑ये॒ नमो॒ नमो॑ निचे॒रवे॑ परिच॒रायार॑ण्यानां॒ पत॑ये॒ नमो॒ नमः॑ सृका॒विभ्यो॒ जिघाग्ं॑सद्भ्यो मुष्ण॒तां पत॑ये॒ नमो॒ नमो॑$सि॒मद्भ्यो॒ नक्त॒ञ्चर॑द्भ्यः प्रकृ॒न्तानां॒ पत॑ये॒ नमो॒ नम॑ उष्णी॒षिने॑ गिरिच॒राय॑ कुलु॒ञ्चानां॒ पत॑ये॒ नमो॒ नम॒ इषु॑मद्भ्यो धन्वा॒विभ्य॑श्च वो॒ नमो॒ नम॑ आतन्-वा॒नेभ्यः॑ प्रति॒दधा॑नेभ्यश्च वो॒ नमो॒ नम॑ आ॒यच्छ॒॑द्भ्यो विसृ॒जद्-भ्य॑श्च वो॒ नमो॒ नमो$स्स॑द्भ्यो॒ विद्य॑द्-भ्यश्च वो॒ नमो॒ नम॒ आसी॑नेभ्यः॒ शया॑नेभ्यश्च वो॒ नमो॒ नमः॑ स्व॒पद्भ्यो॒ जाग्र॑द्-भ्यश्च वो॒ नमो॒ नम॒स्तिष्ठ॑द्भ्यो॒ धाव॑द्-भ्यश्च वो॒ नमो॒ नमः॑ स॒भाभ्यः॑ स॒भाप॑तिभ्यश्च वो॒ नमो॒ नमो॒ अश्वे॒भ्यो$श्व॑पतिभ्यश्च वो॒ नमः॑नम॑ आव्या॒धिनी॓भ्यो वि॒विध्य॑न्तीभ्यश्च वो॒ नमो॒ नम॒ उग॑णाभ्यस्तृगं-ह॒तीभ्यश्च॑ वो॒ नमो॒ नमो॑ गृ॒त्सेभ्यो॑ गृ॒त्सप॑तिभ्यश्च वो॒ नमो॒ नमो॒ व्राते॓भ्यो॒ व्रात॑पतिभ्यश्च वो॒ नमो॒ नमो॑ ग॒णेभ्यो॑ ग॒णप॑तिभ्यश्च वो॒ नमो॒ नमो॒ विरू॑पेभ्यो वि॒श्वरू॑पेभ्यश्च वो॒ नमो॒ नमो॑ मह॒द्भ्यः॑, क्षुल्ल॒केभ्य॑श्च वो॒ नमो॒ नमो॑ र॒थिभ्यो॒$र॒थेभ्य॑श्च वो॒ नमो॒ नमो॒ रथे॓भ्यो॒ रथ॑पतिभ्यश्च वो॒ नमो॒ नमः॑ सेना॓भ्यः सेना॒निभ्य॑श्च वो॒ नमो॒ नमः॑, क्ष॒त्तृभ्यः॑ सङ्ग्रही॒तृभ्य॑श्च वो॒ नमो॒ नम॒स्तक्ष॑भ्यो रथका॒रेभ्य॑श्च वो॒ नमो॑ नमः॒ कुला॑लेभ्यः क॒र्मारे॓भ्यश्च वो॒ नमो॒ नमः॑ पु॒ञ्जिष्टे॓भ्यो निषा॒देभ्य॑श्च वो॒ नमो॒ नमः॑ इषु॒कृद्भ्यो॑ धन्व॒कृद्-भ्य॑श्च वो॒ नमो॒ नमो॑ मृग॒युभ्यः॑ श्व॒निभ्य॑श्च वो॒ नमो॒ नमः॒ श्वभ्यः॒ श्वप॑तिभ्यश्च वो॒ नमः॑ ॥नमो॑ भ॒वाय॑ च रु॒द्राय॑ च॒ नमः॑ श॒र्वाय॑ च पशु॒पत॑ये च॒ नमो॒ नील॑ग्रीवाय च शिति॒कण्ठा॑य च॒ नमः॑ कप॒र्दिने॑ च॒ व्यु॑प्तकेशाय च॒ नमः॑ सहस्रा॒क्षाय॑ च श॒तध॑न्वने च॒ नमो॑ गिरि॒शाय॑ च शिपिवि॒ष्टाय॑ च॒ नमो॑ मी॒ढुष्ट॑माय॒ चेषु॑मते च॒ नमो᳚ ह्र॒स्वाय॑ च वाम॒नाय॑ च॒ नमो॑ बृह॒ते च॒ वर्षी॑यसे च॒ नमो॑ वृ॒द्धाय॑ च सं॒वृद्व॑ने च॒ नमो॒ अग्रि॑याय च प्रथ॒माय॑ च॒ नम॑ आ॒शवे॑ चाजि॒राय॑ च॒ नमः॒ शीघ्रि॑याय च॒ शीभ्या॑य च॒ नम॑ ऊ॒र्म्या॑य चावस्व॒न्या॑य च॒ नमः॑ स्रोत॒स्या॑य च॒ द्वीप्या॑य च ॥ नम ज्ये॒ष्ठाय॑ च कनि॒ष्ठाय॑ च॒ नमः॑ पूर्व॒जाय॑ चापर॒जाय॑ च॒ नमो॑ मध्य॒माय॑ चापग॒ल्भाय॑ च॒ नमो॑ जघ॒न्या॑य च॒ बुध्नि॑याय च॒ नमः॑ सो॒भ्या॑य च प्रतिस॒र्या॑य च॒ नमो॒ याम्या॑य च॒ क्षेम्या॑य च॒ नम॑ उर्व॒र्या॑य च॒ खल्या॑य च॒ नमः॒ श्लोक्या॑य चाऽवसा॒न्या॑य च॒ नमो॒ वन्या॑य च॒ कक्ष्या॑य च॒ नमः॑ श्र॒वाय॑ च प्रतिश्र॒वाय॑ च॒ नम॑ आ॒शुषे॑णाय चा॒शुर॑थाय च॒ नमः॒ शूरा॑य चावभिन्द॒ते च॒ नमो॑ व॒र्मिणे॑ च वरू॒थिने॑ च॒ नमो॑ बि॒ल्मिने॑ च कव॒चिने॑ च॒ नमः॑ श्रु॒ताय॑ च श्रुतसे॒नाय॑ च ॥नमो॑ दुन्दु॒भ्या॑य चाहन॒न्या॑य च॒ नमो॑ धृ॒ष्णवे॑ च प्रमृ॒शाय॑ च॒ नमो॑ दू॒ताय॑ च॒ प्रहि॑ताय च॒ नमो॑ निष॒ङ्गिणे॑ चेषुधि॒मते॑ च॒ नम॑स्ती॒क्ष्णेष॑वे चायु॒धिने॑ च॒ नमः॑ स्वायु॒धाय॑ च सु॒धन्व॑ने च॒ नमः॒ स्रुत्या॑य च॒ पथ्या॑य च॒ नमः॑ का॒ट्या॑य च नी॒प्या॑य च॒ नमः॒ सूद्या॑य च सर॒स्या॑य च॒ नमो॑ ना॒द्याय॑ च वैश॒न्ताय॑ च॒ नमः॒ कूप्या॑य चाव॒ट्या॑य च॒ नमो॒ वर्ष्या॑य चाव॒र्ष्याय॑ च॒ नमो॑ मे॒घ्या॑य च विद्यु॒त्या॑य च॒ नम॑ ई॒ध्रिया॑य चात॒प्या॑य च॒ नमो॒ वात्या॑य च॒ रेष्मि॑याय च॒ नमो॑ वास्त॒व्या॑य च वास्तु॒पाय॑ च ॥नमः॒ सोमा॑य च रु॒द्राय॑ च॒ नम॑स्ता॒म्राय॑ चारु॒णाय॑ च॒ नमः॑ श॒ङ्गाय॑ च पशु॒पत॑ये च॒ नम॑ उ॒ग्राय॑ च भी॒माय॑ च॒ नमो॑ अग्रेव॒धाय॑ च दूरेव॒धाय॑ च॒ नमो॑ ह॒न्त्रे च॒ हनी॑यसे च॒ नमो॑ वृ॒क्षेभ्यो॒ हरि॑केशेभ्यो॒ नम॑स्ता॒राय॒ नम॑श्शं॒भवे॑ च मयो॒भवे॑ च॒ नमः॑ शंक॒राय॑ च मयस्क॒राय॑ च॒ नमः॑ शि॒वाय॑ च शि॒वत॑राय च॒ नम॒स्तीर्थ्या॑य च॒ कूल्या॑य च॒ नमः॑ पा॒र्या॑य चावा॒र्या॑य च॒ नमः॑ प्र॒तर॑णाय चो॒त्तर॑णाय च॒ नम॑ आता॒र्या॑य चाला॒द्या॑य च॒ नमः॒ शष्प्या॑य च॒ फेन्या॑य च॒ नमः॑ सिक॒त्या॑य च प्रवा॒ह्या॑य च ॥ नम॑ इरि॒ण्या॑य च प्रप॒थ्या॑य च॒ नमः॑ किꣳशि॒लाय च॒ क्षय॑णाय च॒ नमः॑ कप॒र्दिने॑ च पुल॒स्तये॑ च॒ नमो॒ गोष्ठ्या॑य च॒ गृह्या॑य च॒ नम॒स्तल्प्या॑य च॒ गेह्या॑य च॒ नमः॑ का॒ट्या॑य च गह्वरे॒ष्ठाय॑ च॒ नमो᳚ ह्रद॒य्या॑य च निवे॒ष्प्या॑य च॒ नमः॑ पाꣳस॒व्या॑य च रज॒स्या॑य च॒ नमः॒ शुष्क्या॑य च हरि॒त्या॑य च॒ नमो॒ लोप्या॑य चोल॒प्या॑य च॒ नम॑ ऊ॒र्व्या॑य च सू॒र्म्या॑य च॒ नमः॑ प॒र्ण्या॑य च पर्णश॒द्या॑य च॒ नमो॑ऽपगु॒रमा॑णाय चाभिघ्न॒ते च॒ नम॑ आख्खिद॒ते च॑ प्रख्खिद॒ते च॒ नमो॑ वः किरि॒केभ्यो॑ दे॒वाना॒ꣳ॒ हृद॑येभ्यो॒ नमो॑ विक्षीण॒केभ्यो॒ नमो॑ विचिन्व॒त्केभ्यो॒ नम॑ आनिर्ह॒तेभ्यो॒ नम॑ आमीव॒त्केभ्यः॑ ॥द्रापे॒ अन्ध॑सस्पते॒ दरि॑द्र॒न्नील॑लोहित । ए॒षां पुरु॑षाणामे॒षां प॑शू॒नां मा भेर्माऽरो॒ मो ए॑षां॒ किंच॒नाम॑मत् । या ते॑ रुद्र शि॒वा त॒नूः शि॒वा वि॒श्वाह॑ भेषजी । शि॒वा रु॒द्रस्य॑ भेष॒जी तया॑ नो मृड जी॒वसे᳚ । इ॒माꣳ रु॒द्राय॑ त॒वसे॑ कप॒र्दिने᳚ क्ष॒यद्वी॑राय॒ प्रभ॑रामहे म॒तिम् ॥ यथा॑ नः॒ शमस॑द्द्वि॒पदे॒ चतु॑ष्पदे॒ विश्वं॑ पु॒ष्टं ग्रामे॑ अ॒स्मिन्नना॑तुरम् । मृ॒डा नो॑ रुद्रो॒त नो॒ मय॑स्कृधि क्ष॒यद्वी॑राय॒ नम॑सा विधेम ते । यच्छं च॒ योश्च॒ मनु॑राय॒जे पि॒ता तद॑श्याम॒ तव॑ रुद्र॒ प्रणी॑तौ । मा नो॑ म॒हान्त॑मु॒त मा नो॑ अर्भ॒कं मा न॒ उक्ष॑न्त-मु॒त मा न॑ उक्षि॒तम् । मा नो॑ऽवधीः पि॒तरं॒ मोत मा॒तरं॑ प्रि॒या मा न॑स्त॒नुवो॑ रुद्र रीरिषः । मान॑स्तो॒के तन॑ये॒ मा न॒ आयु॑षि॒ मा नो॒ गोषु॒ मा नो॒ अश्वे॑षु रीरिषः । वी॒रान्मा नो॑ रुद्र भामि॒तोऽव॑धी-र्ह॒विष्म॑न्तो॒ नम॑सा विधेम ते ॥आ॒रात्ते॑ गो॒घ्न उ॒त पू॑रुष॒घ्ने क्ष॒यद्वी॑राय सु॒म्नम॒स्मे ते॑ अस्तु । रक्षा॑ च नो॒ अधि॑ च देव ब्रू॒ह्यधा॑ च नः॒ शर्म॑ यच्छ द्वि॒बर्हाः᳚ ॥ स्तु॒हि श्रु॒तं ग॑र्त॒सदं॒ युवा॑नं मृ॒गन्न भी॒म-मु॑पह॒त्नुमु॒ग्रम् । मृ॒डा ज॑रि॒त्रे रु॑द्र॒ स्तवा॑नो अ॒न्यन्ते॑ अ॒स्मन्निव॑पन्तु॒ सेनाः᳚ ॥ परि॑णो रु॒द्रस्य॑ हे॒तिर्वृ॑णक्तु॒ परि॑ त्वे॒षस्य॑ दुर्म॒तिर॑घा॒योः । अव॑ स्थि॒रा म॒घव॑द्भ्यस्तनुष्व॒ मीढ्व॑स्तो॒काय॒ तन॑याय मृउडय ॥ मीढु॑ष्टम॒ शिव॑तम शि॒वो नः॑ सु॒मना॑ भव । प॒र॒मे वृ॒क्ष आयु॑धन्नि॒धाय॒ कृत्तिं॒ वसा॑न॒ आच॑र॒ पिना॑कं॒ बिभ्र॒दाग॑हि ॥ विकि॑रिद॒ विलो॑हित॒ नम॑स्ते अस्तु भगवः । यास्ते॑ स॒हस्रꣳ॑ हे॒तयो॒न्यम॒स्मन्निव॑पन्तु॒ ताः ॥ स॒हस्रा॑णि सहस्र॒धा बा॑हु॒वोस्तव॑ हे॒तयः॑ । तासा॒मीशा॑नो भगवः परा॒चीना॒ मुखा॑ कृधि ॥स॒हस्रा॑णि सहस्र॒शो ये रु॒द्रा अधि॒ भूम्या᳚म् । तेषाꣳ॑ सहस्रयोज॒नेऽव॒धन्वा॑नि तन्मसि ॥ अ॒स्मिन् म॑ह॒त्य॑र्ण॒वे᳚ऽन्तरि॑क्षे भ॒वा अधि॑ ॥ नील॑ग्रीवाः शिति॒कण्ठाः᳚ श॒र्वा अ॒धः क्ष॑माच॒राः ॥ नील॑ग्रीवाः शिति॒कण्ठा॒ दिवꣳ॑ रु॒द्रा उप॑श्रिताः ॥ ये वृ॒क्षेषु॑ स॒स्पिंज॑रा॒ नील॑ग्रीवा॒ विलो॑हिताः ॥ ये भू॒ताना॒मधि॑पतयो विशि॒खासः॑ कप॒र्दिनः॑ ॥ ये अन्ने॑षु वि॒विध्य॑न्ति॒ पात्रे॑षु॒ पिब॑तो॒ जनान्॑ ॥ ये प॒थां प॑थि॒रक्ष॑य ऐलबृ॒दा य॒व्युधः॑ ॥ ये ती॒र्थानि॑ प्र॒चर॑न्ति सृ॒काव॑न्तो निष॒ङ्गिणः॑ ॥ य ए॒ताव॑न्तश्च॒ भूयाꣳ॑सश्च॒ दिशो॑ रु॒द्रा वि॑तस्थि॒रे । तेषाꣳ॑ सहस्र-योज॒नेऽव॒धन्वा॑नि तन्मसि ॥ नमो॑ रु॒द्रेभ्यो॒ ये पृ॑थि॒व्यां ऽन्तरि॑क्षे॒ ये दि॒वि येषा॒मन्नं॒ वातो॑ व॒र्षमिष॑व॒स्तेभ्यो॒ दश॒ प्राची॒र्दश॑ दक्षि॒णा दश॑ प्र॒तीची॒र्दशोदी॑चीर्दशो॒र्ध्वास्तेभ्यो॒ नम॒स्ते नो॑ मृडयन्तु॒ ते यं द्वि॒ष्मो यश्च॑ नो॒ द्वेष्टि॒ तं वो॒ जम्भे॑ दधामि ॥'"""


In [ ]:
li = s.split("।")[0:-1]

li = [i+"।" for i in li] 

li =[re.sub("\n","",i) for i in li]

print(len(li))

In [ ]:
emb_li=[]

for i in li:
    emb_li.append(get_sen_emb(i))

print(emb_li[0].shape)

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
t=np.array(emb_li)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:

pca_dim = PCA(n_components=10)
p_dims = pca_dim.fit_transform(t)
pc_kmeans = KMeans(n_clusters=2)
pc_kmeans.fit(p_dims)

In [ ]:
pc_lab =pc_kmeans.labels_
print(pc_lab,"\n")

In [ ]:
pc_cen = pc_kmeans.cluster_centers_

In [ ]:
clus,count = {},0
for i in range(len(t)):
    if (pc_lab[i] == 0):
        clus[count]= (dot(pc_cen[0], p_dims[i])/(norm(pc_cen[0])*norm(p_dims[i])))
    elif(pc_lab[i] ==1):
        clus[count] = (dot(pc_cen[1], p_dims[i])/(norm(pc_cen[1])*norm(p_dims[i])))
    count+=1   

In [ ]:
clus

In [ ]:
a=sorted(clus.items(), key = lambda kv:[kv[1], kv[0]])

In [ ]:
top_sen =[]
for i in range(7):
    top_sen.append(a[i][0])

In [ ]:
to =top_sen.sort()

In [ ]:
to = sorted(top_sen)

In [ ]:
to

In [ ]:
for  i in to:
    print(li[i])

In [ ]:
import shutil
shutil.make_archive('/content/sanskrit_Robert1', 'zip', '/content/sanskrit_bert')

## Training the model

In [ ]:
type(model)

In [ ]:
a=tokenizer.encode("पतत्र श्रीरामंमन पहिता")
model.extract_features(a)

In [ ]:
model(a)

In [ ]:

emb = pipeline('feature-extraction', model='sanskrit_bert', tokenizer='sanskrit_bert')

In [ ]:
te = (emb("भारतदेशः"))

In [ ]:
print(emb("तत्र न"))

In [ ]:
type(te)

In [ ]:
len(te[0])

In [ ]:
te

In [ ]:

sm = pipeline('summarization', model='sanskrit_bert', tokenizer='sanskrit_bert')

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model_st = SentenceTransformer(sanskrit_bert)
ems = model_st.encode("श्रीधरस्वामी कर्णाटकस्य शिवमोग्गमण्डलस्य सागरजनपदस्य स्मीपे वरदपुरम्")
print(ems)

In [ ]:
model.encode